In [104]:

!jt -t monokai

In [16]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
from qiskit.circuit import QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.circuit.library import QFT
from math import gcd, ceil, log2
import numpy as np
from fractions import Fraction
from IPython.display import display
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator
import requests
import random  

## N is limited to 24 bits for now.

In [17]:
def retrieve_denominator(decimal_value, n, N):
    decimal_value = int(decimal_value)
    n = int(n)
    N = int(N)
    
    url = "http://127.0.0.1:5000/denominator"
    payload = {
        "decimal_value": decimal_value,
        "n": n,
        "N": N
    }
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            return response.json().get("denominator")
        else:
            return f"Error: {response.status_code}, {response.json().get('error', 'Unknown error')}"
    except Exception as e:
        return f"Request failed: {str(e)}"


In [30]:
N = 15 # number to factorize
N = int(N)
n = ceil(log2(N))  # number of bits to represent N
q = QuantumRegister(2*n, "q")
t = QuantumRegister(n, "t")
c = ClassicalRegister(2*n, "c")
state = False
service = QiskitRuntimeService()


In [31]:
def Uf(circ):
    for i in range(2*n):
        base = int(a)
        exp = int(2**i)
        mod = int(N)
        mod_exp = pow(base, exp, mod)
        circ.cp(2*np.pi*float(mod_exp)/float(N), q[i], t)

## Ensure denominator.py is running.

In [33]:
while not state:
    a = random.randint(2, N-1)
    
    while gcd(a, N) > 1:
        a = random.randint(2, N-1)
    
    print(f"Chosen base a = {a}")
    
    
    qpe = QuantumCircuit(q, t, c)
    qpe.x(t)
    qpe.barrier()
    qpe.h(q)
    Uf(qpe)
    qpe.append(QFT(2*n, inverse=True), q)
    qpe.measure(range(n*2), range(n*2))
    qpe.barrier()
    qpe.measure(q, c)
    backend = service.least_busy(
        simulator=False,
        operational=True,
        min_num_qubits=2*ceil(log2(N))
    )
    
    compiled = transpile(qpe, backend)
    result = backend.run(compiled, shots=1024).result()
    counts = result.get_counts()
    sorted_measurements = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    
    for measurement, _ in sorted_measurements:
        decimal_value = int(measurement, 2)
        r = retrieve_denominator(decimal_value, n, N)
        
        if isinstance(r, int) and r % 2 == 0:
            x = pow(a, r // 2, N)
            factors = (gcd(x - 1, N), gcd(x + 1, N))
            
            if (factors[0] != 1 or factors[1] != 1) and (factors[0] != N or factors[1] != N):
                if factors[0] * factors[1] == N:
                    print(f"The factors for {N} are: {factors[0]}, {factors[1]}")
                    state = True
                    break
                elif factors[0] != 1 and ((N // factors[0]) * factors[0] == N):
                    print(f"The factors for {N} are: {factors[0]}, {N // factors[0]}")
                    state = True
                    break
                elif factors[1] != 1 and ((N // factors[1]) * factors[1] == N):
                    print(f"The factors for {N} are: {factors[1]}, {N // factors[1]}")
                    state = True
                    break

Chosen base a = 2


C:\Users\HP User\AppData\Local\Temp\ipykernel_15912\4289667536.py:26: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  result = backend.run(compiled, shots=1024).result()


The factors for 15 are: 3, 5
